In [5]:
from matplotlib import pyplot as plt
from systemclass import SNAIL,SNAIL_sweep,Cavity, SNAILC, SNAILC_sweep, SNAILCC, SNAILCC_sweep
import numpy as np
from joblib import Parallel, delayed
import qutip as qt
from qutip import propagator,floquet_modes,Qobj,Options,basis
import numpy as np
def state_index(index,dim):
    m,n,k = index
    M,N,K = dim
    return m*(N*K)+n*K+k
def sort_eigenpairs(eigenvalues, eigenvectors):
    n = eigenvectors.shape[0]
    sorted_indices = []

    for i in range(n):
        max_abs_vals = np.abs(eigenvectors[i, :])
        max_index = np.argmax(max_abs_vals)
        while max_index in sorted_indices:
            max_abs_vals[max_index] = -np.inf
            max_index = np.argmax(max_abs_vals)
        sorted_indices.append(max_index)

    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    return sorted_eigenvalues, sorted_eigenvectors
    
    return sorted_eigenvalues, sorted_eigenvectors
def gaussian_ramp(t0, amplitude):
    """
    Creates a time-dependent function using a Gaussian function that starts near zero,
    peaks at 'amplitude' at time 't0', and remains constant at 'amplitude' for t > t0.

    Args:
    t0 (float): The time at which the function peaks at the amplitude.
    amplitude (float): The maximum amplitude of the Gaussian peak.

    Returns:
    function: A function of time 't' that implements the desired behavior.
    """
    # Define the standard deviation such that the peak is sharp at t0
    sigma = t0 / 3  # Adjust sigma to make the function start close to zero at t=0

    # Define the function using a closure to encapsulate the parameters
    def time_dependent_function(t):
        if t <= t0:
            # Gaussian function centered at t0
            return amplitude * np.exp(-((t - t0)**2) / (2 * sigma**2)) - amplitude * np.exp(-((0 - t0)**2) / (2 * sigma**2))
        else:
            # Return constant amplitude for t > t0
            return amplitude- amplitude * np.exp(-((0 - t0)**2) / (2 * sigma**2))

    return time_dependent_function


flux =0.4325
#snail parameters
EJ = 100*2*np.pi
EC = 0.177*2*np.pi
beta = 0.12
ng = 0
ncut = 300
N = 3
dims = 8
snail = SNAIL(EJ,EC,beta,ng,ncut,flux,N,dims)
Hs,charge_op = snail.spectrum_charge()


#cavity1 parameters
omega_c1 = 4.5*2*np.pi
g_int1 = 0.15/2*(6-omega_c1/2/np.pi)*2*np.pi
dimc1 = 4
cavity1 = Cavity(omega_c1,g_int1,dimc1)

#cavity1 parameters
omega_c2 =  8.5*2*np.pi
g_int2 = 0.15/2*(6-omega_c2/2/np.pi)*2*np.pi
dimc2 = 4
cavity2 = Cavity(omega_c2,g_int2,dimc2)

Hc1, Vc1 = cavity1.hamiltonian()
Hc2, Vc2 = cavity2.hamiltonian()

Ic1 = np.identity(dimc1)
Ic2 = np.identity(dimc2)
Is = np.identity(dims)

Hs = np.kron(np.kron(Hs,Ic1),Ic2)
Hc1 = np.kron(np.kron(Is,Hc1),Ic2)
Hc2 = np.kron(np.kron(Is,Ic1),Hc2)

H_int1 = cavity1.g_int * np.kron(np.kron(charge_op,Vc1),Ic2) * 2 * ((2 * snail.EC / snail.EJ) ** 0.25)
H_int2 = cavity2.g_int * np.kron(np.kron(charge_op,Ic1),Vc2) * 2 * ((2 * snail.EC / snail.EJ) ** 0.25)

H = Hs + Hc1 + Hc2 + H_int1 + H_int2
Hc  =  np.kron(np.kron(charge_op,Ic1),Ic2)
energy0,U = np.linalg.eigh(H)
energy0,U = sort_eigenpairs(energy0, U)
Ud = U.transpose().conjugate()
H = Ud@H@U
Hc = Ud@Hc@U
print(H)
#     snailcc = SNAILCC(snail,cavity1,cavity2)
#     H0,Hc = snailcc.hamiltonian()

#     energy,U = np.linalg.eigh(H0)

index1 = np.argmin(np.abs(energy0 - omega_c1 * np.ones(len(energy0))))
index2 = np.argmin(np.abs(energy0 - omega_c2 * np.ones(len(energy0))))

total_dim = dims*dimc1*dimc2
H0 = Qobj(H)
Hc = Qobj(Hc)
psi0 = qt.basis(total_dim, index1)
Iss = qt.qeye(dims)
s0 = qt.basis(dims, 0)*(qt.basis(dims, 0).dag())
cavity0 = qt.basis(dimc1, 0)*(qt.basis(dimc1, 0).dag())
cavity1 = qt.basis(dimc1, 1)*(qt.basis(dimc1, 1).dag())
cavity2 = qt.basis(dimc1, 2)*(qt.basis(dimc1, 2).dag())
PP1 = qt.Qobj(np.array(qt.tensor(Iss, cavity0, cavity1)))
PP2 = qt.Qobj(np.array(qt.tensor(Iss, cavity1, cavity0)))
omega1 =  6.0 * 2 * np.pi
omega = 4.01*2*np.pi
T = (2*np.pi)/omega
# A1 = 0.01* 2 * np.pi
A = 1.5*2*np.pi
args = {'w': omega,'w1':omega1}
func = gaussian_ramp(4000, A)
func1 = gaussian_ramp(2000, 0.00 * 2 * np.pi)
tlist = np.linspace(0, 15000 * T, 100000)  # Cover ten periods
H = [H0, [Hc, lambda t, args: func(t)*np.cos(args['w']*t)],[Hc, lambda t, args: func1(t)*np.cos(args['w1']*t)]]
#     options = Options(nsteps=1000000)  # Increasing nsteps to 10000, adjust as needed
# result = qt.sesolve(H, psi0, tlist, args=args)

[[-1.13107020e-02-1.34293897e-17j  1.29244964e-15+2.19899557e-15j
  -7.11931192e-18+1.53754691e-16j ...  6.01907545e-21-5.69651304e-20j
  -4.83566698e-21-8.58276575e-22j -6.08286551e-21-5.62189636e-22j]
 [-2.31206113e-17+4.42422249e-16j  5.34280894e+01+5.55111512e-17j
  -2.15881051e-14-5.98972911e-15j ... -6.49681815e-16-6.81839639e-16j
  -6.93277860e-14-5.93899955e-15j -1.54270718e-14+4.69585185e-14j]
 [ 3.46944695e-17+3.46944695e-17j -2.34305847e-14+9.67802227e-15j
   1.06867505e+02+1.11022302e-16j ...  3.32756887e-16-1.14042061e-16j
  -1.68234483e-14-1.60263092e-15j -4.72582013e-15-3.52601298e-15j]
 ...
 [ 1.28047559e-21-3.47415857e-22j -6.49086618e-16+6.82141493e-16j
   3.33087407e-16+1.14531342e-16j ...  3.96545348e+02+0.00000000e+00j
   2.04593287e-14+9.65894031e-15j -3.55271368e-15+1.11022302e-15j]
 [-2.73843312e-21+1.17459647e-21j -6.93277761e-14+5.93908280e-15j
  -1.68240564e-14+1.60238940e-15j ...  1.45890244e-14+1.83186799e-15j
   4.49744968e+02+0.00000000e+00j -9.18015663e-